In [ ]:
# This code was written by Luka Skolc (ETHZ) under the supervision of Krzysztof Barczynski (PMOD/WRC & ETHZ),
# Nils Janitzek (PMOD/WRC & ETHZ) and Louise Harra (PMOD/WRC & ETHZ) in the scope of the ETH Studio 
# Davos Internship in 2023. The work on the code began in March 2023.

In [1]:
# This is the file with the function for plotting of detected objects and writing the data to files. 
# The file is organized into subsections which are separated by a ### line. Below the line are written
# the functions in that subsection.

# The two colormaps are 'sdoaia193' for AIA data and 'sdoaia171' for EUI data

####################################################################################################
####################################################################################################   
# write_data(objects, obj_index, intensities, tol)

def write_data(objects, obj_index, intensities, tol, batch_no):
    # Writes all relevant quantities to a .npy file in the form of a float np.array
    # The explanation of each of the 20 entries is given below.
    
    all_data = np.zeros(20) # Type "float" numpy array for all relevant quantities
    
    all_data[0] = obj_index # Object index as float, should be converted to int for indexing
    
    all_data[1] = len(objects[obj_index][1:]) # Lifetime of the object
    
    all_data[2] = cubic_time(objects[obj_index]) # Cubic time
    
    all_data[3] = cubic_intensity(objects[obj_index], intensities) # Cubic intensity
    
    (minor, major) = inertia(objects[obj_index], intensities, 1, 'max')
    all_data[4] = minor / major # Ratio of principal moments of inertia
    
    traj_raw = object_com_masked_traj(objects[obj_index], intensities, 'expand') # Expansion trajectory
    (ycom_f, xcom_f) = com_footpoint(objects[obj_index], intensities) # Footpoint COM
    
    (traj_rot0, err0) = default_angle_multistep(traj_raw, (ycom_f, xcom_f), 0, tol) # Horizontal trajectory
    
    if err0 == 0:
        (alfa_fit0, beta_fit0, alfa_sig0, rval0) = line_fit(traj_rot0)
        
        all_data[5] = alfa_fit0 # Slope of the horizontal line fit (should be 0)
        all_data[6] = alfa_sig0 # Error of the slope in the horizontal line fit

        (step_ratio1, step_ratio2) = steps_ratio(traj_rot0)
        all_data[7] = step_ratio2 # [0, 1] ratio of forward / backward steps
        
        all_data[8] = steps_dist_ratio(traj_rot0) # [0, 1]

    (traj_rot45, err45) = default_angle_multistep(traj_raw, (ycom_f, xcom_f), np.pi / 4, tol) # slope 1 trajectory
    
    if err45 == 0:
        (alfa_fit45, beta_fit45, alfa_sig45, rval45) = line_fit(traj_rot45)
    
        all_data[9] = alfa_fit45  # Slope of the slope 1 line fit (should be 1)
        all_data[10] = alfa_sig45 # Error of the slope in the slope 1 line fit
        all_data[11] = rval45     # Pearson correlation coefficient of the slope 1 rotated trajectory
    
    all_data[12] = traj_raw.shape[0] # First expansion time = length of the expansion trajectory
    
    all_data[13] = length1(traj_raw) # Distance between the footpoint and the furthest point in the trajectory
    
    all_data[14] = length2(traj_raw) # Largest distance between any two points in the trajectory
    
    all_data[15] = len(time_union(objects[obj_index])) # Size of the object's time union
    
    if err0 == 1:
        all_data[16] = 1
        # 0 means no error, 1 is "err0" = 1, 2 is "err45" = 1, 3 is both "err0" = 1 and "err45" = 1
        
    if err45 == 1:
        all_data[16] = 2
        
    if err45 == 1 and err0 == 1:
        all_data[16] = 3
    
    all_data[17] = traj_raw[0, 0] # y coordinate of the footpoint COM
    all_data[18] = traj_raw[0, 1] # x coordinate of the footpoint COM
    
    # The all_data[19] remains 0
    
    name = make_name(obj_index, 'H', batch_no) + '.npy' # Add a '.npy' suffix to save a numpy array
    np.save(name, all_data)
    
    
####################################################################################################   
# plot_separately_trajectory(objects, obj_index, intensities, traj_type, default, tol, save, batch_no)
# plot_separately_intens(objects, obj_index, intensities, key, vminn, vmaxx, margin, save, ccode, batch_no)
# plot_separately_invivo(objects, obj_index, intensities, stamps, vminn, vmaxx, zoom, margin, save, ccode, batch_no)
# make_video(objects, obj_index, intensities, stamps, vminn, vmaxx, zoom, margin, save, extended, ccode, batch_no)

def plot_separately_trajectory(objects, obj_index, intensities, traj_type, default, tol, save, batch_no): 
    # Plot trajectory of object "objc"
    # "key" specifies whether to use "max" or "sum" methods for calculating the inertia tensor
    # "traj_type" specifies which type of trajectory we want to plot
    # "default" is the angle of the final fitted line
    # "tol" is the slope tolerance when using the "multistep" method to rotate the trajectory

    if traj_type == 'com': # COM of the object, frame by frame
        traj_float = object_com_traj(objects[obj_index], intensities) # to compute the COM and fit a line

    elif traj_type == 'edge': # The most distant pixel from the footpoint, in each frame
        traj_float = object_edge_traj(objects[obj_index], intensities)

    elif traj_type == 'com_masked' or traj_type == 'expand': 
        # "com_masked" COM of the object without the footpoint
        # "expand" COM of the expanding part of the object    ! BEST METHOD !
        traj_float = object_com_masked_traj(objects[obj_index], intensities, traj_type)
    
    # Footpoint's COM
    (ycom_0, xcom_0) = com_footpoint(objects[obj_index], intensities)  

    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    # Plot the original trajectory
    og = ax.scatter(traj_float[:, 1], traj_float[:, 0], s = 40, c = np.arange(1, traj_float.shape[0] + 1, 1), cmap = 'cool', marker='o')
    plt.colorbar(og)
    
    # Rotate the trajectory to the desired angle. Preferrably use the robust "multistep" method.
    (traj_def, err_def) = default_angle_multistep(traj_float, (ycom_0, xcom_0), default, tol) 
    (alfa_fit, beta_fit, alfa_sig, rval) = line_fit(traj_def)
    
    #print('Final slope is {}'.format(alfa_fit))
    
    # Plot the rotated trajectory and the fitted line.
    x_lin = np.linspace(np.amin(traj_float[:, 1]) - 20 - xcom_0, np.amax(traj_float[:, 1]) + 20 - xcom_0, 200)
    ax.plot(x_lin + xcom_0, x_lin * alfa_fit + beta_fit + ycom_0, color = 'k') # plot the fitted line
    ax.scatter(traj_def[:, 1] + xcom_0, traj_def[:, 0] + ycom_0, s = 40, c = np.arange(1, traj_float.shape[0] + 1, 1), edgecolors = 'k', cmap = 'cool', marker = 's')
    
    # Plot the footpoint COM
    ax.scatter(xcom_0, ycom_0, s = 60, marker = 'x', color = 'k')
    
    ax.set_aspect('equal') # x and y ticks are separated by the same amount, no squeezing
    
    ymin = min(np.amin(traj_def[:, 0] + ycom_0), np.amin(traj_float[:, 0])) - 1
    ymax = max(np.amax(traj_def[:, 0] + ycom_0), np.amax(traj_float[:, 0])) + 1
    ax.set_ylim((ymin, ymax)) # Set the plotting range in the y coordinate  
    
    xmin = min(np.amin(traj_def[:, 1]) + xcom_0, np.amin(traj_float[:, 1])) - 1
    xmax = max(np.amax(traj_def[:, 1]) + xcom_0, np.amax(traj_float[:, 1])) + 1
    ax.set_xlim((xmin, xmax)) # Set the plotting range in the x coordinate  
    
    ax.set_xlabel('x [pixels]', fontsize = 15)
    ax.set_ylabel('y [pixels]', fontsize = 15)
    
    ax.set_title('Trajectory of the expanding part', fontsize = 15)
    plt.show()
    
    if save:
        if np.abs(default) < 10**(-7): # Horizontal line
            name = make_name(obj_index, 'F', batch_no)
            
        elif np.abs(default - np.pi / 4) < 10**(-7): # 45-degree line
            name = make_name(obj_index, 'G', batch_no)
            
        plt.savefig(name, bbox_inches = 'tight')
        plt.close()

        
def plot_separately_intens(objects, obj_index, intensities, key, vminn, vmaxx, margin, save, ccode, batch_no): 
    # Plot the "max" or "sum" intensity  of an object in a single picture. "key" specifies whether we 
    # plot the sum of intensities over  all time or if we just take the maximal value over time for 
    # each pixel separately.
    
    (ny, nx, nt) = intensities.shape
    
    array_obj = np.zeros((ny, nx)) # Array in which the intensities will be stored 
    
    if key == 'sum':
        (coordinatess, intenss) = time_sum(objects[obj_index], intensities)
        
    if key == 'max':
        (coordinatess, intenss) = time_max(objects[obj_index], intensities)

    for k in range(len(intenss)):
        (y, x) = coordinatess[k]
        array_obj[y, x] = intenss[k]

    array_obj = np.where(array_obj == 0, vminn, array_obj)
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    if key == 'sum':
        vmax_obj = np.amax(intenss) 
        g = ax.imshow(array_obj, origin = 'lower', norm = colors.LogNorm(vmin = vminn, vmax = vmax_obj), cmap = ccode) 
    
    elif key == 'max':
        g = ax.imshow(array_obj, origin = 'lower', norm = colors.LogNorm(vmin = vminn, vmax = vmaxx), cmap = ccode) 
    
    cbar = plt.colorbar(g)
    if vminn < 200 and vmaxx > 2800:
        cbar.set_ticks([100, 500, 1000, 3000])
        cbar.set_ticklabels([r'$10^2$', r'$5\times 10^2$', r'$10^3$', r'$3\times 10^3$'])
    else:
        cbar.set_ticks([500, 1000, 1500])
        cbar.set_ticklabels([r'$5\times 10^2$', r'$10^3$', r'$1.5\times 10^3$'])
    
    # Plot the COM of the footpoint
    (ycom0, xcom0) = com_footpoint(objects[obj_index], intensities)  
    ax.scatter(xcom0, ycom0, s = 25, marker = 'o', color = 'c')
    
    if key == 'sum':
        ax.set_title('Sum over time of intensities', fontsize = 15)
    
    if key == 'max':
        ax.set_title('Maximum over time of intensities', fontsize = 15)
    
    ax.set_xlabel('x [pixels]', fontsize = 15)
    ax.set_ylabel('y [pixels]', fontsize = 15)
    ax.set_aspect('equal')
    
    (y_lower, y_upper, x_lower, x_upper) = make_limits(objects[obj_index], margin, intensities)
    ax.set_ylim((y_lower, y_upper))
    ax.set_xlim((x_lower, x_upper))
        
    plt.show()
    
    if save:
        name = make_name(obj_index, 'E', batch_no)
        plt.savefig(name, bbox_inches = 'tight')
        plt.close()

    
def plot_separately_invivo(objects, obj_index, intensities, stamps, vminn, vmaxx, zoom, margin, save, ccode, batch_no):
    # Plot an intensity snapshot from when the object was brightest. Mark the footpoint as well.
    maximal_sum = 0
    maximal_index = 0
    
    for i in range(len(objects[obj_index][1:])):
        sum_i = 0
        
        for (yi, xi) in objects[obj_index][1 + i]:
            sum_i += intensities[yi, xi, objects[obj_index][0] + i]
            
        if sum_i > maximal_sum:
            maximal_index = i
            maximal_sum = sum_i
    
    total_max_index = objects[obj_index][0] + maximal_index # Time index of when the object was brightest
    intens_t = intensities[:, :, total_max_index]           # Intensity snapshot
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    g = ax.imshow(intens_t, origin = 'lower', norm = colors.LogNorm(vmin = vminn, vmax = vmaxx), cmap = ccode)
    
    cbar = plt.colorbar(g)
    if vminn < 200 and vmaxx > 2800:
        cbar.set_ticks([100, 500, 1000, 3000])
        cbar.set_ticklabels([r'$10^2$', r'$5\times 10^2$', r'$10^3$', r'$3\times 10^3$'])
    else:
        cbar.set_ticks([500, 1000, 1500])
        cbar.set_ticklabels([r'$5\times 10^2$', r'$10^3$', r'$1.5\times 10^3$'])
    
    (ycom0, xcom0) = com_footpoint(objects[obj_index], intensities)  
    ax.scatter(xcom0, ycom0, s = 25, marker = 'o', color = 'c')
    ax.set_xlabel('x [pixels]', fontsize = 15)
    ax.set_ylabel('y [pixels]', fontsize = 15)
    ax.set_title('Object {} in vivo at '.format(obj_index) + stamps[total_max_index], fontsize = 15)
    
    if zoom:
        (y_lower, y_upper, x_lower, x_upper) = make_limits(objects[obj_index], margin, intensities)
        ax.set_ylim((y_lower, y_upper))
        ax.set_xlim((x_lower, x_upper))
    
    plt.show()
    
    if save:
        if zoom:
            name = make_name(obj_index, 'D', batch_no)
        else: 
            name = make_name(obj_index, 'C', batch_no)
            
        plt.savefig(name, bbox_inches = 'tight')
        plt.close()

        
def make_video(objects, obj_index, intensities, stamps, vminn, vmaxx, zoom, margin, save, extended, ccode, batch_no):
    nt = intensities.shape[2]
    
    if extended: # Create a video that starts earlier and ends later
        t_start = objects[obj_index][0] - 10
        t_stop = objects[obj_index][0] + len(objects[obj_index]) + 10
        # There should be N + 21 frames where N is the object lifetime.
        if t_start < 0:
            t_start = 0
        if t_stop > nt:
            t_stop = nt
    else:
        t_start = objects[obj_index][0] - 2
        t_stop = objects[obj_index][0] + len(objects[obj_index]) + 4
        # There should be N + 7 frames where N is the object lifetime.
        if t_start < 0:
            t_start = 0
        if t_stop > nt:
            t_stop = nt
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    if zoom == False: 
        margin = 60
        
    (y_lower, y_upper, x_lower, x_upper) = make_limits(objects[obj_index], margin, intensities)
    
    img = [None]
    def update_img(i):
        ax.set_title(stamps[i], fontsize = 15) 
        if img[0]: 
            img[0].remove()
        img[0] = ax.imshow(intensities[y_lower : y_upper, x_lower : x_upper, i], norm = colors.LogNorm(vmin = vminn, vmax = vmaxx), origin = 'lower', cmap = ccode, animated=True)

    ani = animation.FuncAnimation(fig, update_img, frames = np.arange(t_start, t_stop, 1), blit = False, repeat = True)
    
    xtik = np.array([j*10 for j in range((x_upper + 10 - x_lower) // 10)])
    ytik = np.array([j*10 for j in range((y_upper + 10 - y_lower) // 10)])
    dely = (np.ceil(y_lower / 10) *10 - y_lower).astype(int) # so that ticks are at 10*N
    delx = (np.ceil(x_lower / 10) *10 - x_lower).astype(int)
    
    ax.set_xticks(xtik + delx, xtik + delx + x_lower)
    ax.set_yticks(ytik + dely, ytik + dely + y_lower)
    ax.set_xlabel('x [pixels]', fontsize = 15)
    ax.set_ylabel('y [pixels]', fontsize = 15)
    plt.show()
    
    if save:
        if zoom:
            name = make_name(obj_index, 'B', batch_no) + '.mp4'
            
        else:
            name = make_name(obj_index, 'A', batch_no) + '.mp4'
            
        writervideo = animation.FFMpegWriter(fps = 5)
        ani.save(name, writer = writervideo)
        plt.close()
    

####################################################################################################
# plot_all_static(objects, intensities, key, vminn, vmaxx, save, ccode, batch_no)
# plot_all_invivo(objects, intensities, vminn, vmaxx, save, ccode, batch_no)
# plot_all_invivo_load(snapshot, classification, properties, ccode, batch_no, save, vminn = 100, vmaxx = 5000)

def plot_all_static(objects, intensities, key, vminn, vmaxx, save, ccode, batch_no): 
    # Plot the intensities of objects in a single picture. "key" specifies whether we plot the sum of 
    # intensities over all time or if we just take the maximal value over time for each pixel 
    # separately. COM's of footpoints are added in teal.
    
    (ny, nx, nt) = intensities.shape
    
    no = len(objects) # Number of objects
    array_obj = np.zeros((ny, nx)) # The array to be plotted
    
    for i in range(no): # Iterate over all objects
        if key == 'sum':
            (coordinates_i, intens_i) = time_sum(objects[i], intensities)
            
        if key == 'max':
            (coordinates_i, intens_i) = time_max(objects[i], intensities)
            
        for k in range(len(intens_i)):
            (y, x) = coordinates_i[k] # "int" type
            array_obj[y, x] = intens_i[k]
        
        # Plot the COMs of the footpoints
        (ycom0i, xcom0i) = com_footpoint(objects[i], intensities)  
        plt.scatter(xcom0i, ycom0i, s = 6, marker = 'o', color = 'c')
        
    array_obj = np.where(array_obj == 0, vminn, array_obj)
    
    if key == 'sum':
        plt.title('Time sum of intensities for all {} objects'.format(no), fontsize = 15)
        vmax_obj = np.amax(array_obj) # Maximum of the summed values
        plt.imshow(array_obj, norm = colors.LogNorm(vmin = vminn, vmax = vmax_obj), origin = 'lower', cmap = ccode)
        
    if key == 'max':
        plt.title('Maximum over time of intensities for all {} objects'.format(no), fontsize = 15)
        plt.imshow(array_obj, norm = colors.LogNorm(vmin = vminn, vmax = vmaxx), origin = 'lower', cmap = ccode)
    
    cbar = plt.colorbar(location = 'bottom')
    if vminn < 200 and vmaxx > 2800:
        cbar.set_ticks([100, 500, 1000, 5000])
        cbar.set_ticklabels([r'$10^2$', r'$5\times 10^2$', r'$10^3$', r'$5\times 10^3$'])
    else:
        cbar.set_ticks([500, 1000, 1500])
        cbar.set_ticklabels([r'$5\times 10^2$', r'$10^3$', r'$1.5\times 10^3$'])
    
    plt.xlabel('x [pixels]', fontsize = 15)
    plt.ylabel('y [pixels]', fontsize = 15)
    plt.show()
    
    if save:
        plt.savefig('all_objects_max' + str(batch_no), bbox_inches = 'tight')
        plt.close()
        

def plot_all_invivo(objects, intensities, vminn, vmaxx, save, ccode, batch_no):
    all_footpoints = np.zeros((len(objects), 2))
    
    for i in range(len(objects)):
        obj_foot = com_footpoint(objects[i], intensities)
        all_footpoints[i, 0] = obj_foot[0]
        all_footpoints[i, 1] = obj_foot[1]

    intens_t = intensities[:, :, int(intensities.shape[2] / 2)] # Intensity snapshot
    
    plt.imshow(intens_t, origin = 'lower', norm = colors.LogNorm(vmin = vminn, vmax = vmaxx), cmap = ccode)
    cbar = plt.colorbar(location = 'bottom')
    if vminn < 200 and vmaxx > 2800:
        cbar.set_ticks([100, 500, 1000, 5000])
        cbar.set_ticklabels([r'$10^2$', r'$5\times 10^2$', r'$10^3$', r'$5\times 10^3$'])
    else:
        cbar.set_ticks([500, 1000, 1500])
        cbar.set_ticklabels([r'$5\times 10^2$', r'$10^3$', r'$1.5\times 10^3$'])
    
    plt.scatter(all_footpoints[:, 1], all_footpoints[:, 0], s = 6, marker = 'o', color = 'c')
    
    plt.xlabel('x [pixels]', fontsize = 15)
    plt.ylabel('y [pixels]', fontsize = 15)
    plt.title('All {} objects in vivo'.format(len(objects)), fontsize = 15)
    plt.show()
    
    if save:
        plt.savefig('all_objects_invivo' + str(batch_no), bbox_inches = 'tight')
        plt.close()

        
def plot_all_invivo_load(snapshot, classification, properties, ccode, batch_no, save, vminn = 100, vmaxx = 5000):
    footpoints_jet_y = []
    footpoints_jet_x = []
    
    footpoints_nj_y = []
    footpoints_nj_x = []
    
    footpoints_other_y = []
    footpoints_other_x = []
    
    for i in range(properties.shape[0]):
        if classification[i] == 2:
            if properties[i, 4] < 1:
                footpoints_other_y.append(properties[i, 17])
                footpoints_other_x.append(properties[i, 18])
                continue
            
        if classification[i] == 1:
            if properties[i, 4] < 1:
                footpoints_jet_y.append(properties[i, 17])
                footpoints_jet_x.append(properties[i, 18])
        
        if classification[i] == 0:
            if properties[i, 4] < 1:
                footpoints_nj_y.append(properties[i, 17])
                footpoints_nj_x.append(properties[i, 18])
    
    plt.imshow(snapshot, origin = 'lower', norm = colors.LogNorm(vmin = vminn, vmax = vmaxx), cmap = ccode)
    cbar = plt.colorbar()
    
    if vminn < 200 and vmaxx > 2800:
        cbar.set_ticks([100, 500, 1000, 5000])
        cbar.set_ticklabels([r'$10^2$', r'$5\times 10^2$', r'$10^3$', r'$5\times 10^3$'])
    else:
        cbar.set_ticks([500, 1000, 1500])
        cbar.set_ticklabels([r'$5\times 10^2$', r'$10^3$', r'$1.5\times 10^3$'])
    
    plt.scatter(footpoints_other_x, footpoints_other_y, s = 6, marker = 'o', color = 'r', label = 'Unclassified')
    plt.scatter(footpoints_nj_x, footpoints_nj_y, s = 6, marker = 'o', color = 'limegreen', label = r'Not jets with $\tilde{I}<0.18$')
    plt.scatter(footpoints_jet_x, footpoints_jet_y, s = 6, marker = 'o', color = 'dodgerblue', label = r'Jets with $\tilde{I}<0.18$')
    
    plt.xlabel('x [pixels]', fontsize = 17)
    plt.ylabel('y [pixels]', fontsize = 17)
    plt.title('All {} brightenings in vivo'.format(properties.shape[0]), fontsize = 17)
    plt.legend(fontsize = 14)
    plt.show()
    
    if save:
        plt.savefig('all_objects_invivo_classified' + str(batch_no), bbox_inches = 'tight')
        plt.close()


####################################################################################################
#annotated_video(objects, intensities, stamps, vminn, vmaxx, ccode, batch_no)

def annotated_video(objects, intensities, stamps, vminn, vmaxx, ccode, batch_no):
    all_footpoints = np.zeros((len(objects), 2))
    
    for i in range(len(objects)):
        obj_foot = com_footpoint(objects[i], intensities)
        all_footpoints[i, 0] = obj_foot[0]
        all_footpoints[i, 1] = obj_foot[1]
    
    (ny, nx, nt) = intensities.shape
    fig = plt.figure()
    ax = fig.add_subplot(111)

    img = [None]
    def update_img(i):
        ax.set_title(stamps[i], fontsize = 15) 
        if img[0]: 
            img[0].remove()
        img[0] = ax.imshow(intensities[:, :, i], norm = colors.LogNorm(vmin = vminn, vmax = vmaxx), origin = 'lower', cmap=ccode, animated=True)
        img[0] = ax.scatter(all_footpoints[:, 1], all_footpoints[:, 0] , s = 6, marker = 'o', color = 'c')
        
    ani = animation.FuncAnimation(fig, update_img, frames = np.arange(nt), blit = False, repeat = True)
    plt.xlabel('x [pixels]', fontsize = 15)
    plt.ylabel('y [pixels]', fontsize = 15)
    plt.show()

    writervideo = animation.FFMpegWriter(fps=5)
    ani.save('whole_video_annotated' + str(batch_no) +'.mp4', writer = writervideo)
    plt.close()
        

####################################################################################################
# animate_trajectories(objects, intensities, traj_type)

def animate_trajectories(objects, intensities, traj_type): # all objects are marked with the same color. The 
    # Pixels are impossible to see otherwise. I could also try to put some special markers
    # this could be done with ax.plot([yo, xo]), see https://stackoverflow.com/questions/45548673
    # /matplotlib-animate-objects-with-different-markers
    
    (ny, nx, nt) = intensities.shape
    array_com = np.zeros((ny, nx, nt)) 
    
    no = len(objects) # number of objects to animate
    
    for i in range(no):
        ti1 = objects[i][0] # initial time of object
        
        if traj_type == 'com': # COM of the object, frame by frame
            trajectory_i = object_com(objects[i], intensities)
            traj_i_rounded = np.rint(trajectory_i).astype(int) # round to integer values and cast to 
            # integer type so it can function as an index. It's not the same as casting float to int!
            for j in range(len(objects[i][1:])):
                array_com[traj_i_rounded[j, 0], traj_i_rounded[j, 1], ti1 + j] = 1
                
        elif traj_type == 'edge': # the most distant pixel from the footpoint
            trajectory_i = edge_path(objects[i], intensities)
            for j in range(len(objects[i][2:])):
                array_com[trajectory_i[j, 0], trajectory_i[j, 1], ti1 + 1 + j] = 1
        
        elif traj_type == 'com_masked' or traj_type == 'expand': 
            # "com_masked" COM without the footpoint
            # "expand" COM of the expanding part of the object
            trajectory_i = np.rint(object_com_masked(objects[i], intensities, traj_type)).astype(int)
            for j in range(trajectory_i.shape[0]):
                array_com[trajectory_i[j, 0], trajectory_i[j, 1], ti1 + 1 + j] = 1
                
    fig1 = plt.figure()
    ax1 = fig1.add_subplot(111)
    ims1 = []
    
    for p in range(nt):
        im1 = ax1.imshow(array_com[:, :, p], vmin = 0, vmax = 1, origin = 'lower', cmap = 'gray', animated = True)
        if p == 0:
            im1 = ax1.imshow(array_com[:, :, p], vmin = 0, vmax = 1, origin = 'lower', cmap = 'gray')
        ims1.append([im1])

    if no != 1:
        plt.title('There are {} objects'.format(no))
    if no == 1:
        plt.title('There is 1 object')
    
    ani = animation.ArtistAnimation(fig1, ims1, interval = 200, blit = False)
    
    return ani


####################################################################################################
# make_histogram(prop, properties, classification, no_bins, dictionary, min_length, max_ratio, separate=True)
# make_2dhistogram(props12, properties, classification, dictionary, min_length, max_ratio)

def make_histogram(prop, properties, classification, no_bins, dictionary, min_length, max_ratio, separate=True):
    histo_jet = [] # jets
    histo_nj = []  # non-jets
    
    for k in range(properties.shape[0]):
        if properties[k, 16]==1 or properties[k, 16]==3: # Non-zero error index -> ignore the object
            continue
            
        if properties[k, 12] < min_length: # Too short first expansion time -> ignore the object
            continue
        
        if properties[k, 4] > max_ratio: # Too large ratio of principal moments -> ignore the object
            continue
        
        if classification[k] == 1: # Jet
            histo_jet.append(properties[k, prop])
            
        if classification[k] == 0: # Not a jet
            histo_nj.append(properties[k, prop])
    
    print('There are {} jets remaining'.format(len(histo_jet)))
    print('There are {} non-jets remaining'.format(len(histo_nj)))
    
    low = np.floor(np.amin(np.array(histo_jet + histo_nj)))
    up = np.ceil(np.amax(np.array(histo_jet + histo_nj)))
    
    #plt.hist([histo_jet, histo_nj], no_bins, (low, up), stacked = True, edgecolor = 'black', linewidth = 0.8, label = ['Jet', 'Not a jet'])
    if separate: # Make separate histograms for jets and non-jets. Can only do this if objects are classified
        plt.hist(histo_nj, no_bins, (low, up), color = 'orange', edgecolor = 'black', linewidth = 0.8, alpha = 0.8, label = 'Not a jet')
        plt.hist(histo_jet, no_bins, (low, up), color = 'blue', edgecolor = 'black', linewidth = 0.8, alpha = 0.5, label = 'Jet')
        plt.legend()
    
    else:
        plt.hist(histo_nj + histo_jet, no_bins, (low, up), color = 'green', edgecolor = 'black', linewidth = 0.8)
    
    plt.title('Histogram for property {}'.format(prop), fontsize = 15)
    plt.xlabel(dictionary[prop], fontsize = 15)
    plt.ylabel('Number of instances', fontsize = 15)
    plt.show()

    
def make_2dhistogram(props12, bins, properties, classification, dictionary, min_length, max_ratio):
    (prop1, prop2) = props12
    (bins1, bins2) = bins
    
    prop1_jet = []
    prop2_jet = []
    
    prop1_nj = []
    prop2_nj = []
    
    for k in range(properties.shape[0]):
        if properties[k, 16]==1 or properties[k, 16]==3: # Non-zero error index -> ignore the object
            continue
        
        if properties[k, 12] < min_length: # Too short first expansion time
            continue
        
        if properties[k, 4] > max_ratio:   # Too large ratio of principal moments
            continue
        
        if classification[k] == 1:
            prop1_jet.append(properties[k, prop1])
            prop2_jet.append(properties[k, prop2])
            
        if classification[k] == 0:
            prop1_nj.append(properties[k, prop1])
            prop2_nj.append(properties[k, prop2])
    
    prop1_jet = np.array(prop1_jet)
    prop2_jet = np.array(prop2_jet)
    prop1_nj = np.array(prop1_nj)
    prop2_nj = np.array(prop2_nj)
    
    min1 = np.floor(min(np.amin(prop1_jet), np.amin(prop1_nj)))
    max1 = np.ceil(max(np.amax(prop1_jet), np.amax(prop1_nj)))
    
    min2 = np.floor(min(np.amin(prop2_jet), np.amin(prop2_nj)))
    max2 = np.ceil(max(np.amax(prop2_jet), np.amax(prop2_nj)))
    
    H_jet, edges1j, edges2j = np.histogram2d(prop1_jet, prop2_jet, bins = [bins1, bins2], range = [[min1, max1], [min2, max2]])
    H_nj, edges1nj, edges2nj = np.histogram2d(prop1_nj, prop2_nj, bins = [bins1, bins2], range = [[min1, max1], [min2, max2]])
    
    fig, axs = plt.subplots(1, 2) 
    
    edges1mesh_j, edges2mesh_j = np.meshgrid(edges1j, edges2j)
    edges1mesh_nj, edges2mesh_nj = np.meshgrid(edges1nj, edges2nj)
    
    H_jet = np.where(H_jet == 0, np.nan, H_jet).T
    H_nj = np.where(H_nj == 0, np.nan, H_nj).T
    
    im0 = axs[0].pcolormesh(edges1mesh_j, edges2mesh_j, H_jet, cmap = 'turbo')
    axs[0].set_title('Jets', fontsize = 15)
    axs[0].set_xlabel(dictionary[prop1], fontsize = 15)
    axs[0].set_ylabel(dictionary[prop2], fontsize = 15)
    plt.colorbar(im0, ax = axs[0])
    
    im1 = axs[1].pcolormesh(edges1mesh_nj, edges2mesh_nj, H_nj, cmap = 'turbo')
    axs[1].set_title('Non-jets', fontsize = 15)
    axs[1].set_xlabel(dictionary[prop1], fontsize = 15)
    axs[1].set_ylabel(dictionary[prop2], fontsize = 15)
    plt.colorbar(im1, ax = axs[1])
    
    plt.show()
    

    